In [1]:
import os
import utils
import numpy as np
import pandas as pd

In [2]:
def read_all_results(dir_path):
    paths = os.listdir(dir_path)
    last_gens = []
    for i in range(len(paths)):
        path = dir_path + paths[i]
        last_gen = sorted(map(lambda x: x.split("_"), 
                        os.listdir(f"{path}/individuals/")), 
                        key=lambda x: int(x[1].split(".")[0]))[-1][-1][:-4]
        print(f"Path: {paths[i]}, last generation: {int(last_gen)}")
        last_gens.append(int(last_gen))
    return paths, last_gens

In [3]:
dir_path = "results/"
paths, last_gens = read_all_results(dir_path)

Path: GA_2021_05_28_00_19_30, last generation: 39
Path: GA_2021_05_28_00_19_55, last generation: 34
Path: GA_2021_05_28_11_00_41, last generation: 31
Path: GA_2021_05_28_00_20_08, last generation: 33
Path: GA_2021_05_28_11_00_56, last generation: 33
Path: GA_2021_05_28_13_16_12, last generation: 32


In [15]:
run = 3
gen = 33
path = dir_path + paths[run]

individuals_path =  f"{path}/individuals/individuals_{gen}.pkl"
score_path       =  f"{path}/final_scores/final_score_{gen}.pkl"
overview_path    =  f"{path}/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [16]:
with open(dir_path + paths[run] + "/run_params.json", "r") as file:
    print(file.read())
    file.close()

{
    "objective": "yll",
    "simulations": 15,
    "process": {
        "horizon": 74,
        "decision_period": 28,
        "policy": "weighted"
    },
    "population_size": 30,
    "random_individuals": true,
    "min_generations": 30
}


In [17]:
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
for ix, i in enumerate(sorted(individuals, key=lambda x: x.mean_score)):
    print(f"{utils.tcolors.BOLD}{ordinal(ix+1)} place:{utils.tcolors.ENDC}")
    print(f"ID: {i.ID}")
    print(f"Score: {np.round(i.mean_score, 3)}")
    
    print(f"Genes:\n{np.round(i.genes, 3)}")
    print()

1st place:
ID: gen_32_004
Score: 18053.0
Genes:
[[[0.134 0.273 0.176 0.251 0.166]
  [0.116 0.231 0.226 0.123 0.304]
  [0.184 0.244 0.291 0.044 0.236]]

 [[0.107 0.041 0.395 0.335 0.123]
  [0.077 0.237 0.386 0.271 0.028]
  [0.29  0.215 0.08  0.277 0.137]]

 [[0.161 0.166 0.295 0.299 0.079]
  [0.122 0.204 0.464 0.157 0.053]
  [0.066 0.137 0.475 0.268 0.053]]]

2nd place:
ID: gen_33_001
Score: 18054.667
Genes:
[[[0.173 0.086 0.297 0.268 0.177]
  [0.163 0.221 0.213 0.118 0.287]
  [0.165 0.241 0.289 0.044 0.261]]

 [[0.106 0.04  0.39  0.331 0.133]
  [0.186 0.209 0.34  0.239 0.025]
  [0.377 0.132 0.075 0.239 0.176]]

 [[0.161 0.165 0.297 0.298 0.08 ]
  [0.122 0.203 0.462 0.159 0.053]
  [0.083 0.144 0.475 0.245 0.053]]]

3rd place:
ID: gen_21_006
Score: 18060.267
Genes:
[[[0.156 0.266 0.171 0.245 0.162]
  [0.1   0.236 0.231 0.126 0.307]
  [0.188 0.244 0.292 0.044 0.232]]

 [[0.107 0.041 0.395 0.335 0.123]
  [0.077 0.237 0.386 0.271 0.028]
  [0.291 0.238 0.058 0.275 0.138]]

 [[0.161 0.165 0.2

In [11]:
print(individuals[0].genes, end="\n"*2)
genes = np.sum(individuals[0].genes, axis=(0,1))
norm = np.sum(genes)
norm_genes = np.divide(genes, norm)
print(norm_genes)

[[[0.26880201 0.03755094 0.45653791 0.03825475 0.19885439]
  [0.21199639 0.15297498 0.19071874 0.18628364 0.25802626]
  [0.28555434 0.12604486 0.18816998 0.26455802 0.1356728 ]]

 [[0.0380807  0.         0.80471269 0.00868688 0.14851973]
  [0.13400663 0.17124373 0.32555228 0.17947985 0.18971751]
  [0.06984915 0.33485728 0.25936679 0.27160597 0.06432081]]

 [[0.24234722 0.11238574 0.27632163 0.0920203  0.27692511]
  [0.07678193 0.152165   0.25492634 0.11165404 0.40447268]
  [0.21303369 0.01044264 0.54098041 0.         0.23554325]]]

[0.17116134 0.1219628  0.3663652  0.12806038 0.21245028]
